In [5]:
import os
import string

import cv2
import numpy as np
from matplotlib import pyplot as plt

from keras import layers
from keras.models import Model
from keras.models import load_model
from keras import metrics
from keras import callbacks

#Init main values
symbols = string.ascii_lowercase + "0123456789" # All symbols captcha can contain
num_symbols = len(symbols)
img_shape = (50, 200, 1)

kernel1 = np.ones((3,3), np.uint8) #used for dilation and erosion
kernel2 = np.ones((3,1), np.uint8) #used for dilation in the vertical

from keras.utils import plot_model


Using TensorFlow backend.


In [2]:
def uAdaptativeThresholding():
    n_samples = len(os.listdir('samples'))
    X = np.zeros((n_samples, 50, 200, 1))
    y = np.zeros((5, n_samples, num_symbols))

    for i, pic in enumerate(os.listdir('samples')):
        
        pic_target = pic[:-4] #tira .jpg
        
        # Read image as grayscale
        img = cv2.imread(os.path.join('samples', pic), cv2.IMREAD_GRAYSCALE)
        
        #print(img)
        
       
        if len(pic_target) < 6:
        
            img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 17, 2) #thresholding
            img = cv2.dilate(img, kernel1, iterations=1)
            img = cv2.erode(img, kernel1, iterations=1)
            img = cv2.dilate(img, kernel2, iterations=1)
        
            # Scale and reshape image
            img = img / 255. #conteúdo de 0 a 1
            img = np.reshape(img, (50, 200, 1)) #muda formato do array para o formato de X
            
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((5, num_symbols))
            
            for j, l in enumerate(pic_target): #percorre cada caracter da string
                
                ind = symbols.find(l)
                targs[j, ind] = 1
            
            X[i] = img
            y[:, i] = targs
    
    # Return final data
    return X, y

In [3]:
def uOtsuThresholding():
    
    n_samples = len(os.listdir('samples'))
    X = np.zeros((n_samples, 50, 200, 1))
    y = np.zeros((5, n_samples, num_symbols))

    for i, pic in enumerate(os.listdir('samples')):
        
        pic_target = pic[:-4] #tira .jpg
        
        # Read image as grayscale
        img = cv2.imread(os.path.join('samples', pic), cv2.IMREAD_GRAYSCALE)
        
        if len(pic_target) < 6:
        
            ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) #OtsuThresholding
            img = cv2.dilate(img, kernel1, iterations=1)
            img = cv2.erode(img, kernel1, iterations=1)
            img = cv2.dilate(img, kernel2, iterations=1)

            '''if pic_target == '226md':
                plt.imshow(img, 'gray')'''

            # Scale and reshape image
            img = img / 255. #conteúdo de 0 a 1
            img = np.reshape(img, (50, 200, 1)) #muda formato do array
            
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((5, num_symbols))
            
            for j, l in enumerate(pic_target):
                
                ind = symbols.find(l)
                targs[j, ind] = 1
            
            X[i] = img
            y[:, i] = targs
    
    # Return final data
    return X, y

In [4]:
def uBlurOtsuThresholding():
    n_samples = len(os.listdir('samples'))
    X = np.zeros((n_samples, 50, 200, 1))
    y = np.zeros((5, n_samples, num_symbols))

    for i, pic in enumerate(os.listdir('samples')):
        
        pic_target = pic[:-4] #tira .jpg
        
        # Read image as grayscale
        img = cv2.imread(os.path.join('samples', pic), cv2.IMREAD_GRAYSCALE)
       
        if len(pic_target) < 6:
            
            blur = cv2.GaussianBlur(img, (5, 5), 0)
            ret, img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) #blurOtsuThresholding
            img = cv2.dilate(img, kernel1, iterations=1)
            img = cv2.erode(img, kernel1, iterations=1)
            img = cv2.dilate(img, kernel2, iterations=1)
            
            # Scale and reshape image
            img = img / 255. #conteúdo de 0 a 1
            img = np.reshape(img, (50, 200, 1)) #muda formato do array
            
            # Define targets and code them using OneHotEncoding
            targs = np.zeros((5, num_symbols))
            
            for j, l in enumerate(pic_target):
                
                ind = symbols.find(l)
                targs[j, ind] = 1
            
            X[i] = img
            y[:, i] = targs
    
    # Return final data
    return X, y

In [3]:
# Define a function that creates a net
def create_net():
    
    img = layers.Input(shape=img_shape) # Get image as an input and process it through some Convs
    
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img)
    
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 100x25
    
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 50x13
    
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    
    bn = layers.BatchNormalization()(conv3)
    
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 25x7
    
    # Get flattened vector and make 5 branches from it. Each branch will predict one letter
    flat = layers.Flatten()(mp3) #array 25*7=175
    
    outs = []
    for _ in range(5):
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1)
        res = layers.Dense(num_symbols, activation='sigmoid')(drop)

        outs.append(res)
    
    # Compile model and return it
    model = Model(img, outs)
    model.compile('rmsprop', loss=['categorical_crossentropy', 'categorical_crossentropy',
                                   'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'])
    return model

In [6]:
def predictAT(filepath):
    
    #print(filepath)
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    #print(type(img))
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 17, 2) #thresholding
    img = cv2.dilate(img, kernel1, iterations=1)
    img = cv2.erode(img, kernel1, iterations=1)
    img = cv2.dilate(img, kernel2, iterations=1) / 255.
    
    res = np.array(net.predict(img[np.newaxis, :, :, np.newaxis]))
    
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
        
    return capt, sum(probs) / 5

In [7]:
def predictOT(filepath):
    
    #print(filepath)
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    #print(type(img))
    ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) #OtsuThresholding
    img = cv2.dilate(img, kernel1, iterations=1)
    img = cv2.erode(img, kernel1, iterations=1)
    img = cv2.dilate(img, kernel2, iterations=1)/ 255.
    res = np.array(net.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
        
    return capt, sum(probs) / 5

In [8]:
def predictBOT(filepath):
    
    #print(filepath)
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    #print(type(img))
    blur = cv2.GaussianBlur(img, (5, 5), 0)
    ret, img = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) #blurOtsuThresholding
    img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 17, 2) #thresholding
    img = cv2.dilate(img, kernel1, iterations=1)
    img = cv2.erode(img, kernel1, iterations=1)
    img = cv2.dilate(img, kernel2, iterations=1)/ 255.
    res = np.array(net.predict(img[np.newaxis, :, :, np.newaxis]))
    ans = np.reshape(res, (5, 36))
    l_ind = []
    probs = []
    for a in ans:
        l_ind.append(np.argmax(a))
        probs.append(np.max(a))

    capt = ''
    for l in l_ind:
        capt += symbols[l]
        
    return capt, sum(probs) / 5

In [9]:
def accuracy(y_pred, y_true):
    
    total = len(y_pred)
    rigth = 0
    
    for i in range(0, total):
        
        if y_pred[i] == y_true[i]:
            rigth += 1
            print('pred: ' + y_pred[i] + ' true: '+ y_true[i])
            
    perc = rigth/total
    
    return perc

In [10]:
X, y = uAdaptativeThresholding()
X_train, y_train = X[:970], y[:, :970]
X_test, y_test = X[970:], y[:, 970:]

net = create_net()

history = net.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], 
                  batch_size=32, epochs=30, validation_split=0.2)

test = os.listdir('samples')[970:]

y_true = []
y_pred = []

for ele in test:
    
    y_target = ele[:-4]
    
    if len(y_target) < 6: #evita esse arquivo: samples/.ipynb_checkpoints <class 'NoneType'>
    
        y_true.append(y_target) 
        #print(ele)

        pred, prob = predictAT('samples/'+ele)
        y_pred.append(pred)
        
        #print(pred, prob)
    
#print(y_true[-1])

print('Evaluate: ', net.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]]))
print('Accuracy: ', accuracy(y_pred, y_true))

Train on 776 samples, validate on 194 samples
Epoch 1/30
776/776 [==============================] - 14s 18ms/step - loss: 17.6168 - dense_2_loss: 3.4887 - dense_4_loss: 3.5208 - dense_6_loss: 3.6273 - dense_8_loss: 3.5118 - dense_10_loss: 3.4681 - val_loss: 16.2331 - val_dense_2_loss: 3.1247 - val_dense_4_loss: 3.2613 - val_dense_6_loss: 3.3650 - val_dense_8_loss: 3.2599 - val_dense_10_loss: 3.2223
Epoch 2/30
776/776 [==============================] - 13s 16ms/step - loss: 15.7922 - dense_2_loss: 3.0841 - dense_4_loss: 3.1000 - dense_6_loss: 3.3121 - dense_8_loss: 3.1893 - dense_10_loss: 3.1067 - val_loss: 16.7809 - val_dense_2_loss: 3.2560 - val_dense_4_loss: 3.2973 - val_dense_6_loss: 3.4166 - val_dense_8_loss: 3.4871 - val_dense_10_loss: 3.3239
Epoch 3/30
776/776 [==============================] - 11s 15ms/step - loss: 14.5163 - dense_2_loss: 2.7154 - dense_4_loss: 2.8638 - dense_6_loss: 3.0164 - dense_8_loss: 2.9729 - dense_10_loss: 2.9479 - val_loss: 13.5396 - val_dense_2_loss: 2.

Epoch 24/30
776/776 [==============================] - 11s 14ms/step - loss: 0.8388 - dense_2_loss: 0.0896 - dense_4_loss: 0.0994 - dense_6_loss: 0.1815 - dense_8_loss: 0.2646 - dense_10_loss: 0.2037 - val_loss: 3.3897 - val_dense_2_loss: 0.0051 - val_dense_4_loss: 0.5708 - val_dense_6_loss: 1.1338 - val_dense_8_loss: 0.6409 - val_dense_10_loss: 1.0391
Epoch 25/30
776/776 [==============================] - 12s 15ms/step - loss: 0.7619 - dense_2_loss: 0.0782 - dense_4_loss: 0.1573 - dense_6_loss: 0.1622 - dense_8_loss: 0.1974 - dense_10_loss: 0.1668 - val_loss: 3.4530 - val_dense_2_loss: 0.0316 - val_dense_4_loss: 0.5189 - val_dense_6_loss: 1.0877 - val_dense_8_loss: 0.7217 - val_dense_10_loss: 1.0932
Epoch 26/30
776/776 [==============================] - 12s 15ms/step - loss: 0.6790 - dense_2_loss: 0.0608 - dense_4_loss: 0.0940 - dense_6_loss: 0.1659 - dense_8_loss: 0.2101 - dense_10_loss: 0.1482 - val_loss: 3.2816 - val_dense_2_loss: 0.0238 - val_dense_4_loss: 0.5056 - val_dense_6_los

In [11]:
X, y = uOtsuThresholding()
X_train, y_train = X[:970], y[:, :970]
X_test, y_test = X[970:], y[:, 970:]

net = create_net()
history = net.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], 
                  batch_size=32, epochs=30, validation_split=0.2)

test = os.listdir('samples')[970:]

y_true = []
y_pred = []

for ele in test:
    
    y_target = ele[:-4]
    
    if len(y_target) < 6: #evita esse arquivo: samples/.ipynb_checkpoints <class 'NoneType'>
    
        y_true.append(y_target) 
        #print(ele)

        pred, prob = predictOT('samples/'+ele)
        y_pred.append(pred)
        
        #print(pred, prob)
    
#print(y_true[-1])

print('Evaluate: ', net.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]]))
print('Accuracy: ', accuracy(y_pred, y_true))

Train on 776 samples, validate on 194 samples
Epoch 1/30
776/776 [==============================] - 13s 17ms/step - loss: 17.9458 - dense_12_loss: 3.4938 - dense_14_loss: 3.6286 - dense_16_loss: 3.6807 - dense_18_loss: 3.4846 - dense_20_loss: 3.6581 - val_loss: 16.1578 - val_dense_12_loss: 3.1814 - val_dense_14_loss: 3.2800 - val_dense_16_loss: 3.3733 - val_dense_18_loss: 3.1277 - val_dense_20_loss: 3.1954
Epoch 2/30
776/776 [==============================] - 11s 14ms/step - loss: 16.4312 - dense_12_loss: 3.2443 - dense_14_loss: 3.3430 - dense_16_loss: 3.3360 - dense_18_loss: 3.2197 - dense_20_loss: 3.2881 - val_loss: 15.8120 - val_dense_12_loss: 3.1516 - val_dense_14_loss: 3.1899 - val_dense_16_loss: 3.2130 - val_dense_18_loss: 3.0963 - val_dense_20_loss: 3.1612
Epoch 3/30
776/776 [==============================] - 11s 15ms/step - loss: 15.6341 - dense_12_loss: 3.1077 - dense_14_loss: 3.1161 - dense_16_loss: 3.1942 - dense_18_loss: 3.1284 - dense_20_loss: 3.0878 - val_loss: 16.6807 - 

In [12]:
X, y = uBlurOtsuThresholding()
X_train, y_train = X[:970], y[:, :970]
X_test, y_test = X[970:], y[:, 970:]

net = create_net()
history = net.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], 
                  batch_size=32, epochs=30, validation_split=0.2)

test = os.listdir('samples')[970:]

y_true = []
y_pred = []

for ele in test:
    
    y_target = ele[:-4]
    
    if len(y_target) < 6: #evita esse arquivo: samples/.ipynb_checkpoints <class 'NoneType'>
    
        y_true.append(y_target) 
        #print(ele)

        pred, prob = predictBOT('samples/'+ele)
        y_pred.append(pred)
        
        #print(pred, prob)
    
#print(y_true[-1])

print('Evaluate: ', net.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]]))
print('Accuracy: ', accuracy(y_pred, y_true))

Train on 776 samples, validate on 194 samples
Epoch 1/30
776/776 [==============================] - 14s 18ms/step - loss: 17.6310 - dense_22_loss: 3.5828 - dense_24_loss: 3.5406 - dense_26_loss: 3.5529 - dense_28_loss: 3.4830 - dense_30_loss: 3.4718 - val_loss: 16.0097 - val_dense_22_loss: 3.2638 - val_dense_24_loss: 3.2882 - val_dense_26_loss: 3.2127 - val_dense_28_loss: 3.1553 - val_dense_30_loss: 3.0896
Epoch 2/30
776/776 [==============================] - 11s 14ms/step - loss: 15.8197 - dense_22_loss: 3.2070 - dense_24_loss: 3.1464 - dense_26_loss: 3.1398 - dense_28_loss: 3.1561 - dense_30_loss: 3.1703 - val_loss: 14.5676 - val_dense_22_loss: 2.8725 - val_dense_24_loss: 2.9535 - val_dense_26_loss: 2.9424 - val_dense_28_loss: 2.8878 - val_dense_30_loss: 2.9115
Epoch 3/30
776/776 [==============================] - 12s 15ms/step - loss: 14.7404 - dense_22_loss: 2.8988 - dense_24_loss: 2.8653 - dense_26_loss: 2.9601 - dense_28_loss: 2.9389 - dense_30_loss: 3.0772 - val_loss: 13.7000 - 